In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys

In [4]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.utils import to_categorical

In [11]:
# text_to_word_sequence() 함수

#전처리 할 텍스트트
text = '해보지 않으면 해낼 수 없다. 당신이 선택한 의사결정을 후회하지 말라'

#해당 텍스트를 토큰화화 문장을 단어 단위로 토큰화 해줌줌
result = text_to_word_sequence(text)
print('원문:\n ', text)
print('\n토큰화:\n ', result)

원문:
  해보지 않으면 해낼 수 없다. 당신이 선택한 의사결정을 후회하지 말라

토큰화:
  ['해보지', '않으면', '해낼', '수', '없다', '당신이', '선택한', '의사결정을', '후회하지', '말라']


In [18]:
#단어 빈도 세기 
#전처리 하려는 세 개 의 문장 정하기

docs = ['먼저 텍스트의 각 단어를 나누어 토큰화 합니다.',
'텍스트의 단어로 토큰화해야 딥러닝에서 인식됩니다.',
'토큰화한 결과는 딥러닝에서 사용할 수 있습니다.',
]

#텍스트의 토큰화 (전처리)

token=Tokenizer() #토큰화 함수 사용
token.fit_on_texts(docs) #토큰화 함수를 문장에 적용

#단어 빈도수 계산 > 각 옵션에 맞게 출력

print('단어 카운트:\n', token.word_counts) #단어의 빈도 세어 줌
print('\n문장 카운트:\n', token.document_count) #문장장의 빈도 세어 줌
print('\n각 단어가 몇 개의 문장에 포함되어 있는가:\n', token.word_docs) 
print('\n각 단어에 매겨진 인덱스 값:\n', token.word_index) 

단어 카운트:
 OrderedDict([('먼저', 1), ('텍스트의', 2), ('각', 1), ('단어를', 1), ('나누어', 1), ('토큰화', 1), ('합니다', 1), ('단어로', 1), ('토큰화해야', 1), ('딥러닝에서', 2), ('인식됩니다', 1), ('토큰화한', 1), ('결과는', 1), ('사용할', 1), ('수', 1), ('있습니다', 1)])

문장 카운트:
 3

각 단어가 몇 개의 문장에 포함되어 있는가:
 defaultdict(<class 'int'>, {'단어를': 1, '먼저': 1, '합니다': 1, '각': 1, '텍스트의': 2, '토큰화': 1, '나누어': 1, '인식됩니다': 1, '딥러닝에서': 2, '토큰화해야': 1, '단어로': 1, '사용할': 1, '결과는': 1, '있습니다': 1, '토큰화한': 1, '수': 1})

각 단어에 매겨진 인덱스 값:
 {'텍스트의': 1, '딥러닝에서': 2, '먼저': 3, '각': 4, '단어를': 5, '나누어': 6, '토큰화': 7, '합니다': 8, '단어로': 9, '토큰화해야': 10, '인식됩니다': 11, '토큰화한': 12, '결과는': 13, '사용할': 14, '수': 15, '있습니다': 16}


In [19]:
#단어의 원-핫 인코딩

text = '오랫동안 꿈꾸는 이는 그 꿈을 닮아간다'
token = Tokenizer()
token.fit_on_texts([text])
print(token.word_index)

{'오랫동안': 1, '꿈꾸는': 2, '이는': 3, '그': 4, '꿈을': 5, '닮아간다': 6}


In [25]:
x=token.texts_to_sequences([text])
x

[[1, 2, 3, 4, 5, 6]]

In [26]:
#index +1 1 원핫 인코딩 배열 만들기 앞에 0이 있어야 하니 인덱스 숫자 맞춰야해서서 +1 꼭 추가

word_size = len(token.word_index) +1
word_size

7

In [27]:
to_categorical(x, num_classes=word_size)

array([[[0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 1.]]], dtype=float32)

In [32]:
#텍스트를 읽고 긍정 부정 예측하기
docs = ["너무 재밌네요","최고예요","참 잘 만든 영화예요","추천하고 싶은 영화입니다","한번 더 보고싶네요","글쎄요","별로예요","생각보다 지루하네요","연기가 어색해요","재미없어요"]

In [34]:
#긍정 리뷰 1 부정 리뷰 0

classes = np.array([1,1,1,1,1,0,0,0,0,0])

In [35]:
#토큰화

token = Tokenizer()
token.fit_on_texts(docs)
print(token.word_index)

{'너무': 1, '재밌네요': 2, '최고예요': 3, '참': 4, '잘': 5, '만든': 6, '영화예요': 7, '추천하고': 8, '싶은': 9, '영화입니다': 10, '한번': 11, '더': 12, '보고싶네요': 13, '글쎄요': 14, '별로예요': 15, '생각보다': 16, '지루하네요': 17, '연기가': 18, '어색해요': 19, '재미없어요': 20}


In [36]:
x=token.texts_to_sequences(docs)
x #토큰화 결과

[[1, 2],
 [3],
 [4, 5, 6, 7],
 [8, 9, 10],
 [11, 12, 13],
 [14],
 [15],
 [16, 17],
 [18, 19],
 [20]]

In [38]:
#단어의 길이를 맞추어 주어야 함 > padding (zero padding)
#x라는 문장의 4로 길이를 맞추어줘

pad_sequences(x, 4)

array([[ 0,  0,  1,  2],
       [ 0,  0,  0,  3],
       [ 4,  5,  6,  7],
       [ 0,  8,  9, 10],
       [ 0, 11, 12, 13],
       [ 0,  0,  0, 14],
       [ 0,  0,  0, 15],
       [ 0,  0, 16, 17],
       [ 0,  0, 18, 19],
       [ 0,  0,  0, 20]], dtype=int32)

In [41]:
padded_x = pad_sequences(x, 4)
print('패딩 결과:\n', padded_x)

패딩 결과:
 [[ 0  0  1  2]
 [ 0  0  0  3]
 [ 4  5  6  7]
 [ 0  8  9 10]
 [ 0 11 12 13]
 [ 0  0  0 14]
 [ 0  0  0 15]
 [ 0  0 16 17]
 [ 0  0 18 19]
 [ 0  0  0 20]]


In [47]:
#임베딩에 입력될 단어의 수 지정

word_size = len(token.word_index) +1
#인덱스는 0부터 시작하기 때문에 길이를 맞춰줌 (+1)

#단어 임베딩 포함, 딥러닝 모델 만들기

model = Sequential()
model.add(Embedding(word_size, 16, input_length=4))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.summary()
#8에 인덱스 20 곱한다믄 8 더해주기
#차원 축소 다음 32+1

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 4, 16)             336       
                                                                 
 flatten_2 (Flatten)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 401
Trainable params: 401
Non-trainable params: 0
_________________________________________________________________


In [48]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(padded_x, classes, epochs=20)
print('\n ACC: %.4f' %(model.evaluate(padded_x, classes)[1]))

Epoch 1/20
1/1 [==============================] - 1s 934ms/step - loss: 0.6956 - accuracy: 0.4000
Epoch 2/20
1/1 [==============================] - 0s 12ms/step - loss: 0.6925 - accuracy: 0.5000
Epoch 3/20
1/1 [==============================] - 0s 11ms/step - loss: 0.6894 - accuracy: 0.5000
Epoch 4/20
1/1 [==============================] - 0s 16ms/step - loss: 0.6864 - accuracy: 0.5000
Epoch 5/20
1/1 [==============================] - 0s 16ms/step - loss: 0.6833 - accuracy: 0.6000
Epoch 6/20
1/1 [==============================] - 0s 11ms/step - loss: 0.6803 - accuracy: 0.7000
Epoch 7/20
1/1 [==============================] - 0s 12ms/step - loss: 0.6773 - accuracy: 0.8000
Epoch 8/20
1/1 [==============================] - 0s 12ms/step - loss: 0.6742 - accuracy: 0.8000
Epoch 9/20
1/1 [==============================] - 0s 17ms/step - loss: 0.6712 - accuracy: 0.9000
Epoch 10/20
1/1 [==============================] - 0s 11ms/step - loss: 0.6682 - accuracy: 0.9000
Epoch 11/20
1/1 [===========

In [28]:
from tensorflow.keras.layers import Embedding

In [ ]:
#단어 임베딩
